In [1]:
import nltk
import string
import os
import re
import numpy as np
import pandas as pd
from glob import glob
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
ids = []
docs = []

for f in glob('./ydb/subtitles/*.txt'):
    with open(f) as fh:
        docs.append(fh.read())
        ids.append(f)

In [3]:
# preprocessing
docs = [re.sub('(<[^>]+>|</[^>]+>|\[[^\]]*\])', '', d.replace('\n', '')) for d in docs]

In [4]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(docs)

In [5]:
def get_top_n(csr_matrix, row, n = 10):
    # https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices
    row = csr_matrix.getrow(row).toarray()[0].ravel()
    top_indicies = row.argsort()[::-1][:n]
    top_values = row[row.argsort()[::-1][:n]]
    
    return top_indicies, top_values

In [6]:
features = vectorizer.get_feature_names()

data = []

for i in range(len(docs)):
    top = [features[x] for x in get_top_n(X, i, 10)[0]]
    link = 'https://youtu.be/' + ids[i][-18:-7]
    title = ids[i][12:-19]
    data.append({ 'title': title, 'link': link, 'keywords': ','.join(top) })

In [7]:
df = pd.DataFrame(data)

In [8]:
df.to_excel('top_keywords_from_subtitles.xlsx')

In [9]:
df.to_json('top_keywords_from_subtitles.json')